<a href="https://www.kaggle.com/code/antongalysh/transfer-learning?scriptVersionId=173749057" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

# Імпорт бібліотек

In [ ]:
import torch
from torchvision import datasets, transforms
device = "cuda" if torch.cuda.is_available() else "cpu"

## Створення ImageFolder

In [ ]:
data_dir = '/kaggle/input/bone-break-classification-image-dataset/Bone Break Classification/Bone Break Classification'

# Створіть екземпляр ImageFolder
dataset = datasets.ImageFolder(root=data_dir)

**Пояснення:**

* Імпортуємо необхідні бібліотеки: `torch` для основних операцій PyTorch та `datasets` і `transforms` з `torchvision`.
* `datasets.ImageFolder` використовується для створення екземпляру, що представляє набір даних.
* Аргумент `root` вказує кореневий каталог, що містить теки ваших класів.

**Розуміння атрибутів об'єкта ImageFolder:**

* **`classes`:** Цей атрибут є списком, що містить назви класів в алфавітному порядку, які відповідають назвам папок у вашому наборі даних.
* **`class_to_idx`:** Цей словник зіставляє назви класів (ключі) з відповідними цілочисельними індексами (значеннями).
* **`imgs`:** Цей список містить кортежі, де кожен кортеж представляє зображення та відповідну мітку класу (індекс).

In [ ]:
dataset.classes

**Розбиття набору даних для навчання та валідації:**

* Поділ вашого набору даних на навчальний та валідаційний набори має вирішальне значення для оцінювання моделі. 
* Функція `random_split` у PyTorch дозволяє випадковим чином розділити набір даних у потрібних пропорціях.

In [ ]:
from torch.utils.data import random_split

train_ratio = 0.8

# Розділіть набір даних
train_data, test_data = random_split(dataset, [train_ratio, 1-train_ratio])

# Зміна transformer після поділу

In [ ]:
train_transform = transforms.Compose([
    transforms.Resize((224, 224)), # Зміна розміру зображення до 256x256 пікселів
    transforms.RandomHorizontalFlip(p=0.5), # Випадково перевернути по горизонталі з ймовірністю 50%
    transforms.ToTensor(), # Перетворити зображення у тензори PyTorch
    transforms.Normalize(mean=[0.3517, 0.3557, 0.3570],
                         std=[0.2325, 0.2347, 0.2353]) # нормалізація для моделей
])

test_transform = transforms.Compose([
    transforms.Resize((224, 224)), # Зміна розміру зображення до 256x256 пікселів
    transforms.ToTensor(), # Перетворити зображення у тензори PyTorch
    transforms.Normalize(mean=[0.3517, 0.3557, 0.3570],
                         std=[0.2325, 0.2347, 0.2353]) # нормалізація для моделей
])


class TransformDataset(torch.utils.data.Dataset):
    def __init__(self, subset, transform=None):
        self.subset = subset
        self.transform = transform
        
    def __getitem__(self, index):
        x, y = self.subset[index]
        if self.transform:
            x = self.transform(x)
        return x, y
        
    def __len__(self):
        return len(self.subset)

    
train_data = TransformDataset(train_data, transform = train_transform)
test_data = TransformDataset(test_data, transform = test_transform)

**Пояснення:**

* Визначаємо бажане співвідношення розбиття (`train_ratio`) для навчальних даних.
* Функція `random_split` отримує набір даних та список довжин як аргументи. Довжини визначають кількість вибірок для кожного розбиття.
* Створює два нових набори даних (`train_data` та `val_data`), що представляють навчальну та валідаційну вибірки відповідно.

**5. Створення завантажувачів даних:**

* PyTorch's `DataLoader` допомагає керувати ефективним завантаженням даних під час навчання. Він дозволяє пакетно завантажувати дані, перемішувати зразки (необов'язково) і обробляти багатопроцесорні дані (необов'язково) для пришвидшення навчання.

Обчислити параметри для зображень можна і вручну, якщо у вас специфічні зображення

In [ ]:
means = []
stds = []
for img, _ in train_data:
    means.append(torch.mean(img, [1, 2]).tolist())
    stds.append(torch.std(img, [1, 2]).tolist())

mean = torch.mean(torch.tensor(means), [0])
std = torch.mean(torch.tensor(stds), [0])

mean, std

In [ ]:
img.shape

# Мій випадок

In [ ]:
# train_data = datasets.ImageFolder(root=data_dir, transform=train_transform,
#                                   is_valid_file=lambda path: 'Train' in path)

# test_data = datasets.ImageFolder(root=data_dir, transform=test_transform,
#                                  is_valid_file=lambda path: 'Test' in path)

In [ ]:
batch_size = 256

# Створіть завантажувачі даних
train_loader = torch.utils.data.DataLoader(train_data, shuffle=True, batch_size=batch_size)
test_loader = torch.utils.data.DataLoader(test_data, shuffle=True, batch_size=batch_size)

# Трансферне навчання

Трансферне навчання - це потужний метод глибокого навчання, який дозволяє використовувати знання, отримані від попередньо навченої моделі на великому наборі даних, для вирішення нової, пов'язаної з нею задачі. Уявіть, що ви навчили величезну модель ідентифікувати тисячі об'єктів. Навчання з перенесенням дозволяє використовувати цю попередньо навчену модель як відправну точку для нового завдання, наприклад, класифікації різних типів квітів. Повторно використовуючи вивчені ознаки з попередньо навченої моделі, ви можете досягти хороших результатів у новому завданні з меншою кількістю даних і меншим часом навчання порівняно з навчанням моделі з нуля.

**Переваги навчання з перенесенням:**

* **Скорочення часу навчання:** Попередньо навчені моделі вже вивчили потужні представлення ознак з великих наборів даних. Це економить час і обчислювальні ресурси при застосуванні до нових завдань.
* **Покращена продуктивність:** Трансферне навчання часто дозволяє досягти кращої точності на нових завданнях, особливо з обмеженими даними, порівняно з навчанням моделі з нуля.
* **Ефективна розробка моделей:** Трансферне навчання дозволяє будувати складні моделі навіть з меншими наборами даних, прискорюючи процес розробки.

![](https://www.researchgate.net/publication/342400905/figure/fig4/AS:905786289057792@1592967688003/The-architecture-of-our-transfer-learning-model.jpg)

# **Популярні попередньо навчені моделі для комп'ютерного зору:**

* **ImageNet:** Великий набір даних з мільйонами мічених зображень у тисячах категорій об'єктів. Популярні моделі, навчені на ImageNet, включають:
* **VGG (Very Deep Convolutional Neural Network):** Класична архітектура з глибокими згортковими шарами для вилучення ознак.
* **ResNet (залишкова мережа):** Вирішує проблему зникаючого градієнта в глибоких мережах, що призводить до кращої продуктивності.
* **DenseNet (щільно зв'язана згорткова мережа):** Покращує розповсюдження ознак для складних задач з меншою кількістю параметрів.

![](https://glassboxmedicine.com/wp-content/uploads/2020/12/vgg-resnet-googlenet-1.png?w=1024)

# [Моделі для різних задач](https://pytorch.org/vision/0.9/models.html#torchvision-models)

# **Заморожування проти тонкого налаштування параметрів:**

* **Заморожування параметрів:** Передбачає, що ваги (параметри) попередньо навченої моделі не підлягають навчанню під час процесу перенесення. Це гарантує, що основні шари виділення ознак залишаються незмінними і фокусує навчання на кінцевих шарах, адаптованих до нового завдання.
* **Точне налаштування:** Передбачає, що деякі або всі параметри попередньо навченої моделі можуть бути навчені під час навчання з перенесенням. Це дозволяє моделі адаптувати вивчені функції до нової задачі, використовуючи при цьому попередньо набуті знання. Точне налаштування зазвичай застосовується до останніх шарів попередньо навченої моделі, ближче до виходу.

# [Посилання на документацію по моделях](https://pytorch.org/vision/stable/models.html#classification)

In [ ]:
from torchvision import models

vgg19 = models.vgg19_bn(pretrained=True)
vgg19

In [ ]:
from torch import nn
import torch.nn.functional as F
import numpy as np


class TransferLearningClassifier(nn.Module):
    def __init__(self, num_classes=10):
        super().__init__()

        vgg = models.vgg19_bn(pretrained=True)
        
        # від'єднання градієнтів
        for param in vgg.parameters():
            param.requires_grad = False
        
        # кількість нейронів на виході
        in_features = vgg.classifier[0].in_features
        
        # деактивація останнього шару
        vgg.classifier = nn.Identity()
        
        # створення потрібних шарів
        self.feature_extractor = vgg
        
        self.dropout = nn.Dropout(0.2)
        self.linear = nn.Linear(in_features, num_classes)
        

    def forward(self, x):
        out = self.feature_extractor(x) # (batch, in_features)
        
        out = self.dropout(out)
        out = self.linear(out)
        
        return out


    def predict(self, X, device='cpu'):
        X = torch.FloatTensor(np.array(X)).to(device)

        with torch.no_grad():
            y_pred = F.softmax(self.forward(X), dim=-1)

        return y_pred.cpu().numpy()


model = TransferLearningClassifier(len(dataset.classes)).to(device)

In [ ]:
!pip install -q torchsummary

In [ ]:
from torchsummary import summary

summary(model, input_size=(3, 224, 224))

In [ ]:
# @title Функція для тренування
import time

def train(model, optimizer, loss_fn, train_dl, val_dl,
          metrics=None, metrics_name=None, epochs=20, device='cpu', task='regression'):
    '''
    Runs training loop for classification problems. Returns Keras-style
    per-epoch history of loss and accuracy over training and validation data.

    Parameters
    ----------
    model : nn.Module
        Neural network model
    optimizer : torch.optim.Optimizer
        Search space optimizer (e.g. Adam)
    loss_fn :
        Loss function (e.g. nn.CrossEntropyLoss())
    train_dl :
        Iterable dataloader for training data.
    val_dl :
        Iterable dataloader for validation data.
    metrics: list
        List of sklearn metrics functions to be calculated
    metrics_name: list
        List of matrics names
    epochs : int
        Number of epochs to run
    device : string
        Specifies 'cuda' or 'cpu'
    task : string
        type of problem. It can be regression, binary or multiclass

    Returns
    -------
    Dictionary
        Similar to Keras' fit(), the output dictionary contains per-epoch
        history of training loss, training accuracy, validation loss, and
        validation accuracy.
    '''

    print('train() called: model=%s, opt=%s(lr=%f), epochs=%d, device=%s\n' % \
          (type(model).__name__, type(optimizer).__name__,
           optimizer.param_groups[0]['lr'], epochs, device))

    metrics = metrics if metrics else []
    metrics_name = metrics_name if metrics_name else [metric.__name__ for metric in metrics]

    history = {} # Collects per-epoch loss and metrics like Keras' fit().
    history['loss'] = []
    history['val_loss'] = []
    for name in metrics_name:
        history[name] = []
        history[f'val_{name}'] = []

    start_time_train = time.time()

    for epoch in range(epochs):

        # --- TRAIN AND EVALUATE ON TRAINING SET -----------------------------
        start_time_epoch = time.time()

        model.train()
        history_train = {name: 0 for name in ['loss']+metrics_name}

        for batch in train_dl:
            x    = batch[0].to(device)
            y    = batch[1].to(device)
            y_pred = model(x)
            loss = loss_fn(y_pred, y)

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            y_pred = y_pred.detach().cpu().numpy()
            y = y.detach().cpu().numpy()


            history_train['loss'] += loss.item() * x.size(0)
            for name, func in zip(metrics_name, metrics):
                try:
                    history_train[name] += func(y, y_pred) * x.size(0)
                except:
                    if task == 'binary': y_pred_ = y_pred.round()
                    elif task == 'multiclass': y_pred_ = y_pred.argmax(axis=-1)
                    history_train[name] += func(y, y_pred_) * x.size(0)

        for name in history_train:
            history_train[name] /= len(train_dl.dataset)


        # --- EVALUATE ON VALIDATION SET -------------------------------------
        model.eval()
        history_val = {'val_' + name: 0 for name in metrics_name+['loss']}

        with torch.no_grad():
            for batch in val_dl:
                x    = batch[0].to(device)
                y    = batch[1].to(device)
                y_pred = model(x)
                loss = loss_fn(y_pred, y)

                y_pred = y_pred.cpu().numpy()
                y = y.cpu().numpy()

                history_val['val_loss'] += loss.item() * x.size(0)
                for name, func in zip(metrics_name, metrics):
                    try:
                        history_val['val_'+name] += func(y, y_pred) * x.size(0)
                    except:
                        if task == 'binary': y_pred_ = y_pred.round()
                        elif task == 'multiclass': y_pred_ = y_pred.argmax(axis=-1)

                        history_val['val_'+name] += func(y, y_pred_) * x.size(0)

        for name in history_val:
            history_val[name] /= len(val_dl.dataset)

        # PRINTING RESULTS

        end_time_epoch = time.time()

        for name in history_train:
            history[name].append(history_train[name])
            history['val_'+name].append(history_val['val_'+name])

        total_time_epoch = end_time_epoch - start_time_epoch

        print(f'Epoch {epoch+1:4d} {total_time_epoch:4.0f}sec', end='\t')
        for name in history_train:
            print(f'{name}: {history[name][-1]:10.3g}', end='\t')
            print(f"val_{name}: {history['val_'+name][-1]:10.3g}", end='\t')
        print()

    # END OF TRAINING LOOP

    end_time_train       = time.time()
    total_time_train     = end_time_train - start_time_train
    print()
    print('Time total:     %5.2f sec' % (total_time_train))

    return history

In [ ]:
# Визначення функції втрат та оптимізатора

loss_fn = nn.CrossEntropyLoss()

# Оптимізатор (Adam) для оновлення ваг моделі
optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)

In [ ]:
from sklearn.metrics import accuracy_score

history = train(model, optimizer, loss_fn, train_loader, test_loader,
                epochs=50,
                metrics=[accuracy_score],
                device=device,
                task='multiclass')

In [ ]:
import matplotlib.pyplot as plt

def plot_metric(history, name):
    plt.title(f"Model results with {name}")
    plt.plot(history[name], label='train')
    plt.plot(history['val_'+name], label='val')
    plt.xlabel('Epoch')
    plt.ylabel(name)
    plt.legend()


plot_metric(history, 'loss')

In [ ]:
plot_metric(history, 'accuracy_score')

In [ ]:
from sklearn.metrics import ConfusionMatrixDisplay

model = model.to('cpu')  # відключаємо від gpu

loader = torch.utils.data.DataLoader(test_data, batch_size=len(test_data))
X_test, y_test = next(iter(loader))

y_pred = model.predict(X_test)

ConfusionMatrixDisplay.from_predictions(y_test, y_pred.argmax(-1), display_labels=dataset.classes)
plt.xticks(rotation=90)
plt.plot()

In [ ]:
from sklearn.metrics import classification_report

print(classification_report(y_test, y_pred.argmax(-1), target_names=dataset.classes))